## Loading the libraries

In [11]:
import pandas as pd
import numpy as np

import requests
from requests import get
from bs4 import BeautifulSoup

from time import sleep
from random import randint

## Scraping from the web page

In [12]:
# Creating the lists we want to write into
titles = []
years = []
time = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []

In [13]:
# Getting English translated titles from the movies
headers = {'Accept-Language': 'en-US, en;q=0.5'}

There are 1000 movies and each page has 50 movies listed.

So the first 50 movies' url: https://www.imdb.com/search/title/?groups=top_1000&ref_=adv_prv

Movies from 51 to 100: https://www.imdb.com/search/title/?groups=top_1000&start=51&ref_=adv_nxt

Movies from 101 to 150: https://www.imdb.com/search/title/?groups=top_1000&start=101&ref_=adv_nxt

In [14]:
pages = np.arange(1, 1001, 50)
pages

array([  1,  51, 101, 151, 201, 251, 301, 351, 401, 451, 501, 551, 601,
       651, 701, 751, 801, 851, 901, 951])

In [16]:
# Storing each of the urls of 50 movies 
for page in pages:
    # Getting the contents from the each url
    response = requests.get('https://www.imdb.com/search/title/?groups=top_1000&start=' + str(page) + '&ref_=adv_nxt', headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Aiming the part of the html we want to get the information from
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    
    # Controling the loop’s rate by pausing the execution of the loop for a specified amount of time
    # Waiting time between requests for a number between 2-10 seconds
    sleep(randint(2,10))
    
    for container in movie_div:
        # Extracting the movie's name
        h3 = container.find('h3', class_='ipc-title__text')
        if h3:
            name = h3.text.strip()  # Using strip() to remove leading/trailing whitespaces
            titles.append(name)
        else:
            print("No 'h3' tag with class 'ipc-title__text' found in container")
            titles.append("N/A")  # Append "N/A" or similar if no matching 'h3' tag is found

        # Scraping the movie's year
        year = container.h3.find('span', class_='lister-item-year').text if container.h3.find('span', class_='lister-item-year') else 'N/A'
        years.append(year)

        # Scraping the movie's length
        runtime = container.find('span', class_='runtime').text if container.find('span', class_='runtime') else 'N/A'
        time.append(runtime)

        # Scraping the rating
        imdb_rating = container.strong.text if container.strong else 'N/A'
        imdb_ratings.append(imdb_rating)

        # Scraping the metascore
        m_score = container.find('span', class_='metascore').text.strip() if container.find('span', class_='metascore') else 'N/A'
        metascores.append(m_score)

        # Scraping votes and gross earnings
        nv = container.find_all('span', attrs={'name': 'nv'})
        vote = nv[0].text if nv and len(nv) > 0 else 'N/A'
        votes.append(vote)
        grosses = nv[1].text if nv and len(nv) > 1 else 'N/A'
        us_gross.append(grosses)


## Creating the Data Set

In [17]:
movies = pd.DataFrame({'movie':titles,
                       'year':years,
                       'time_minute':time,
                       'imdb_rating':imdb_ratings,
                       'metascore':metascores,
                       'vote':votes,
                       'gross_earning':us_gross})

movies.head()

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning


In [18]:
movies.dtypes

movie            float64
year             float64
time_minute      float64
imdb_rating      float64
metascore        float64
vote             float64
gross_earning    float64
dtype: object

## Cleaning the Data Set

In [8]:
# Cleaning 'year' column
movies['year'] = movies['year'].str.extract('(\d+)').astype(int)
movies.head(3)

AttributeError: Can only use .str accessor with string values!

In [ ]:
# Cleaning 'time_minute' column
movies['time_minute'] = movies['time_minute'].str.extract('(\d+)').astype(int)
movies.head(3)

In [ ]:
# Cleaning 'metascore' column
movies['metascore'] = movies['metascore'].str.extract('(\d+)')
# convert it to float and if there are dashes turn it into NaN
movies['metascore'] = pd.to_numeric(movies['metascore'], errors='coerce')

In [ ]:
# Cleaning 'vote' column
movies['vote'] = movies['vote'].str.replace(',', '').astype(int)
movies.head(3)

In [ ]:
# Cleaning 'gross_earning' column
# left strip $ and right strip M 
movies['gross_earning'] = movies['gross_earning'].map(lambda x: x.lstrip('$').rstrip('M'))
# convert it to float and if there are dashes turn it into NaN
movies['gross_earning'] = pd.to_numeric(movies['gross_earning'], errors='coerce')
movies.head(3)

In [ ]:
movies.dtypes

## Final Data Set

In [ ]:
movies

## Saving the data to a csv file

In [ ]:
movies.to_csv('movies.csv')